In [1]:
import requests
import datetime
import csv
import os
import subprocess
import cv2
import urllib
import pandas as pd
import numpy as np
import soundfile as sf

token = "951280115:AAFDywEsju5xY4UguZ4pmGTcGSyidkEb058"
xml_path1 = "haarcascade_frontalface_default.xml"
chat_ids = pd.read_csv('chat_ids.csv')
ffmpeg_path = './bin/ffmpeg'

class BotHandler:
    def __init__(self, token):
        self.token = token
        self.api_url = "https://api.telegram.org/bot{}/".format(token)
        
    def get_updates(self, offset=None, timeout=30):  
        response = requests.get(self.api_url + 'getUpdates', data={'timeout': timeout, 'offset': offset})
        return response.json()['result']
    
    def get_last_update(self):  
        results = self.get_updates()
        if len(results) > 0:
            last_update = results[-1]
        else:
            last_update = []
        return last_update
    
    def getFile(self, file_id):
        response = requests.get(self.api_url + 'getFile', data={'file_id': file_id})
        return response.json()['result']
    
    def downloadImageFile(self,detector,file_path,chat_id):
        response = urllib.request.urlopen("https://api.telegram.org/file/bot"+self.token+"/"+file_path)
        arr = np.asarray(bytearray(response.read()), dtype=np.uint8)
        img = cv2.imdecode(arr, -1)
        faces_coord = detector.detect(img)
        print(len(faces_coord))
        if len(faces_coord) > 0:
            dest_filename = './' + str(chat_id) + '/photo' + file_path[11:-4] + '.png'
            cv2.imwrite(dest_filename, img)
    
    def downloadAudioFile(self, file_path, chat_id):
        response = requests.get("https://api.telegram.org/file/bot"+self.token+"/"+file_path, allow_redirects=True)
        open('./' + str(chat_id) + '/temp_audio_message.oga', 'wb').write(response.content)
        
        src_filename = './' + str(chat_id) + '/temp_audio_message.oga'
        dest_filename = './' + str(chat_id) + '/audio_message' + file_path[11:-4] + '.wav'
        process = subprocess.run([ffmpeg_path, '-i', src_filename, '-ar', '16000', dest_filename])
    
    def send_msg(self, chat_id, text):  
        params = {'chat_id': chat, 'text': text}
        response = requests.post(self.api_url + 'sendMessage', data=params)
        return response
    
    def add_user(self, chat_id, chat_ids):
        path = "/bots/" + str(chat_id)
        try:
            with open('chat_ids.csv','a') as file:
                new_row = [chat_id, str(datetime.datetime.now())]
                writer = csv.writer(file)
                writer.writerow(new_row)
            
            os.mkdir(path)
            return pd.read_csv('chat_ids.csv')
        
        except OSError:
            return chat_ids

In [2]:
class FaceDetector(object):
    def __init__(self, xml_path):
        self.classifier = cv2.CascadeClassifier(xml_path)
    
    def detect(self, image):
        scale_factor = 1.2
        min_neighbors = 5
        min_size = (30, 30)
        faces_coord = self.classifier.detectMultiScale(image,
                                                       scaleFactor=scale_factor,
                                                       minNeighbors=min_neighbors,
                                                       minSize=min_size,
                                                       flags=cv2.CASCADE_SCALE_IMAGE)
        return faces_coord

In [ ]:
bot = BotHandler(token) 
detector = FaceDetector(xml_path1)
now = datetime.datetime.now()

def main():  
    new_offset = None
    today = now.day
    hour = now.hour

    while True:
        bot.get_updates(new_offset)

        last_update = bot.get_last_update()
        if len(last_update) > 0:
            msg_keys = list(last_update['message'].keys())

            last_update_id = last_update['update_id']
            last_chat_id = last_update['message']['chat']['id']
            last_chat_name = last_update['message']['chat']['first_name']

            global chat_ids
            if last_chat_id not in list(chat_ids['chat_id']):
                chat_ids = bot.add_user(last_chat_id, chat_ids)
                
            if 'text' in msg_keys:
                last_chat_text = last_update['message']['text']
                print(last_chat_text)
            elif 'voice' in msg_keys:
                last_chat_voice = last_update['message']['voice']
                audio_file_path = bot.getFile(last_chat_voice['file_id'])['file_path']
                bot.downloadAudioFile(audio_file_path, last_chat_id)
            elif 'photo' in msg_keys:
                last_chat_image = last_update['message']['photo']
                image_file_path = bot.getFile(last_chat_image[-1]['file_id'])['file_path']
                bot.downloadImageFile(detector, image_file_path, last_chat_id)
                
            new_offset = last_update_id + 1

if __name__ == '__main__':  
    try:
        main()
    except KeyboardInterrupt:
        exit()

['message_id', 'from', 'chat', 'date', 'photo']
0
['message_id', 'from', 'chat', 'date', 'photo']
1
['message_id', 'from', 'chat', 'date', 'photo']
0
